In [ ]:
import pandas as pd
import numpy as np
import nltk 
from bs4 import BeautifulSoup            
import re
import time
import ast
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from collections import Counter
import math
from textblob import TextBlob as tb
from sklearn.pipeline import Pipeline


In [ ]:
nameTrainCSV = 'trainWithListOfCleanWords'
nameTestCSV = 'testWithListOfCleanWords'

train = pd.read_csv('../data/processed/' + nameTrainCSV + '.csv', encoding='utf-8')
train['BagOfWords'] = dict
train.head(10)


In [ ]:
start = time.time()
for x in range(len(train)):
    train.set_value(col='listOfCleanWords',
                index=x,
                value=ast.literal_eval(train["listOfCleanWords"][x]))
    train.set_value(col='BagOfWords',
                index=x,
                value=Counter(train["listOfCleanWords"][x]))
end = time.time()
print(end - start)

In [ ]:
train.head()

In [ ]:
def getDataFrameFilterByToxicClass(df, toxicName, cls = 1):
    return train[train[toxicName] == cls]['listOfCleanWords']

toxicName = 'toxic'

clnTxtWrds = getDataFrameFilterByToxicClass(train, toxicName).tolist()
clnTxtWrdsNoToxic = getDataFrameFilterByToxicClass(train, toxicName, 0).tolist()

allTextsToxic = [' '.join(clnTxtWrds[x]) for x in range(len(clnTxtWrds))]
allTextsNoToxic = [' '.join(clnTxtWrdsNoToxic[x]) for x in range(len(clnTxtWrdsNoToxic))]

In [ ]:
allTextsToxic[:10]

In [ ]:
allTextsNoToxic[:10]

In [ ]:
topNFeatures = 500
tdidfVectTrainToxic = TfidfVectorizer(max_features=topNFeatures)
matrixTdidfTrainToxic = tdidfVectTrainToxic.fit_transform(allTextsToxic)

featureArrayToxicTrain = np.array(tdidfVectTrainToxic.get_feature_names())
tfidfOrderedToxicTrain = np.argsort(matrixTdidfTrainToxic.toarray()).flatten()[::-1]


tdidfVectTrainNoToxic = TfidfVectorizer(max_features=topNFeatures)
matrixTdidfTrainNoToxic = tdidfVectTrainNoToxic.fit_transform(allTextsNoToxic)

featureArrayNoToxicTrain = np.array(tdidfVectTrainNoToxic.get_feature_names())
tfidfOrderedNoToxicTrain = np.argsort(matrixTdidfTrainNoToxic.toarray()).flatten()[::-1]




In [ ]:
selectedWordsTrainToxic = featureArrayToxicTrain[tfidfOrderedToxicTrain][:topNFeatures]
selectedWordsTrainNoToxic = featureArrayNoToxicTrain[tfidfOrderedNoToxicTrain][:topNFeatures]
selectedWordsTrain = []
for x in range(topNFeatures):
    if len(selectedWordsTrain) <= topNFeatures: 
        selectedWordsTrain = np.unique(np.append(selectedWordsTrain, np.unique([selectedWordsTrainNoToxic[x], selectedWordsTrainToxic[x]])))    
selectedWordsTrain = selectedWordsTrain[:topNFeatures]


In [ ]:
len(selectedWordsTrain)

In [ ]:
# With all selected toxic words get BoW from all Texts 

clnTxtWrds = train['listOfCleanWords']
allTexts = [' '.join(clnTxtWrds[x]) for x in range(len(clnTxtWrds))]

vectTrain = CountVectorizer(vocabulary=selectedWordsTrain)
matrixBoWTrain = vectTrain.fit_transform(allTexts).toarray()

In [ ]:
matrixBoWTrain.shape

In [ ]:
matrixBoWTrain[6]

In [ ]:
pd.DataFrame(matrixBoWTrain, columns=selectedWordsTrain).head()

In [ ]:
matrixBoWTrain

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
X = matrixBoWTrain
y = np.array(train['toxic'])
Nfold = 5
skf = StratifiedKFold(n_splits=Nfold, shuffle=True)
skf.get_n_splits(X, y)
meanAcc = 0.0
for train_index, test_index in skf.split(X, y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    nb = MultinomialNB()
    %time nb.fit(X_train, y_train)
    y_pred = nb.predict(X_test)
    acc = metrics.accuracy_score(y_test, y_pred)
    meanAcc += acc
    print (acc)

print("La precisión media es de:", np.round(meanAcc / Nfold, 4))



In [ ]:
test = pd.read_csv('../data/processed/' + nameTestCSV + '.csv', encoding='utf-8')
test.head()

In [ ]:
clnTxtWrdsTest = test['listOfCleanWords'].tolist()
allTextsTest = [' '.join(clnTxtWrdsTest[x]) for x in range(len(clnTxtWrdsTest))]

In [ ]:
vectTest = CountVectorizer(vocabulary=selectedWordsTrain)
matrixBoWTest = vectTest.fit_transform(allTextsTest).toarray()

In [ ]:
matrixBoWTest.shape

In [ ]:
X_test = matrixBoWTest

In [ ]:
nb = MultinomialNB()
%time nb.fit(X, y)
y_pred = nb.predict(X_test)


In [ ]:
np.where(y_pred > 0)

In [ ]:
X.shape